# AES in CBC and AES in CTR
- In this project you will implement two encryption/decryption systems, one using AES in CBC mode and another using AES in counter mode (CTR). In both cases the 16-byte encryption IV is chosen at random and is prepended to the ciphertext.
- For CBC encryption we use the PKCS5 padding scheme discussed  While we ask that you implement both encryption and decryption, we will only test the decryption function. In the following questions you are given an AES key and a ciphertext (both are hex encoded ) and your goal is to recover the plaintext and enter it in the input boxes provided below.
- For an implementation of AES you may use an existing crypto library such as PyCrypto (Python). While it is fine to use the built-in AES functions, we ask that as a learning experience you implement CBC and CTR modes yourself.

## Q1
```
CBC key: 140b41b22a29beb4061bda66b6747e14
CBC Ciphertext 1:
4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81
```

## Q2
```
CBC key: 140b41b22a29beb4061bda66b6747e14
CBC Ciphertext 2:
5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253
```

## Q3
```
CTR key: 36f18357be4dbd77f050515c73fcf9f2
CTR Ciphertext 1:
69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329
```

## Q4
```
CTR key: 36f18357be4dbd77f050515c73fcf9f2
CTR Ciphertext 2:
770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451
```

# Misc Notes 
- LIBRARY USED
  - https://cryptography.io
  - https://github.com/pyca/cryptography
- INTERESTING RESOURCES
  - https://cryptopals.com
  - https://www.crypto101.io
  - https://www.dlitz.net/software/pycrypto/doc
- AES 
  - AES (Advanced Encryption Standard) is a block cipher standardized by NIST. AES is both fast, and cryptographically strong. It is a good default choice for encryption.
  - Parameters: key (bytes-like) – The secret key. This must be kept secret. Either 128, 192, or 256 bits long.
- bytes-like
    - A bytes-like object contains binary data and supports the Buffer Protocol
    - https://docs.python.org/3/c-api/buffer.html
    - Includes bytes, bytearray, memoryview objects

In [1]:
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers import modes, Cipher
from cryptography.hazmat.backends import default_backend
from os import urandom

In [13]:
# if it can display the byte in ascii it will, 
# else it will display it in hex IE. \x00 to \xff
# example fromhex("414243FF41") will be displayed as b'ABC0xffA'

test1 = bytes.fromhex("414243FF41")
test2 = bytes.fromhex("FF") 
test3 = urandom(1)
test4 = urandom(3)
print(test1)
print(test2)
print(test3)
print(test4)

print(test1.hex())

def xor_bytes(x, y):
    return bytes(a ^ b for a, b in zip(x, y))

m = bytes.fromhex("FF00FF")
k = bytes.fromhex("FFFFFF") 

print(xor_bytes(m, k)) #00FF00

m = bytearray.fromhex("FF00FF")
k = bytearray.fromhex("FFFFFF") 

print(xor_bytes(m, k).hex()) #00FF00

b'ABC\xffA'
b'\xff'
b'\xb6'
b'\xda\x05\x01'
414243ff41
b'\x00\xff\x00'
00ff00


In [3]:
SECRET_MESSAGE = bytearray("a secret message", "utf-8")
key = urandom(32)
iv = urandom(16)

# -------------------------------------
# EXAMPLE: CBC-AES ENCRYPTION DECRYTION
# -------------------------------------
cipher = Cipher(AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(SECRET_MESSAGE) + encryptor.finalize()

decryptor = cipher.decryptor()
plaintext = decryptor.update(ciphertext) + decryptor.finalize()

print("SECRET MESSAGE: \n", SECRET_MESSAGE)
print("KEY: \n", key)
print("IV: \n", iv)
print("CIPHERTEXT: ", ciphertext)
print("PLAINTEXT: \n", plaintext)

SECRET MESSAGE: 
 bytearray(b'a secret message')
KEY: 
 b'\x0b\x82I\x87\t\xc1\xaai\x0f\xfdy\xa0\xc8\xe3\xcc\xa6[\x98\xd1\xc3\x06;\xbcs\x93x\xb07\x047\x96a'
IV: 
 b'\x1b9\x91\x88dxP\x11\xa3\x97\xe5#;\xe1\x0e1'
CIPHERTEXT:  b'&\x10\xb2r\x92&tz\x1a0\x81J\x85\xca\x0c&'
PLAINTEXT: 
 b'a secret message'


In [4]:
def debug_decrypt(key_string, key, ivct_string, iv, ct, plaintext):
    print("key_string: ", key_string)
    print("key_string length in characters: ", len(key_string))
    print("key: ", key)
    print("key length in bytes: ", len(key))
    print("ivct_string:", ivct_string)
    print("ivct_string length in characters:", len(ivct_string))
    print("IV in bytearray: ", iv)
    print("CT in bytearray: ", ct)
    print("PLAINTEXT: \n", plaintext)

In [5]:
# -------------------------------------
# EXAMPLE: CBC-AES DECRYTION
# -------------------------------------
   
def decrypt(ivct_string, key_string, mode_function, bytes_per_block=16, debug=False):
    
    key = bytes.fromhex(key_string)    
    ivct = bytearray.fromhex(ivct_string)
    iv = ivct[0:bytes_per_block]
    ct = ivct[bytes_per_block:]

    cipher = Cipher(AES(key), mode_function(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ct) + decryptor.finalize()

    if debug is True:
        debug_decrypt(key_string, key, ivct_string, iv, ct, plaintext)

    return plaintext 

In [6]:
key_string =  "140b41b22a29beb4061bda66b6747e14"
ivct_string = "4ca00ff4c898d61e1edbf1800618fb28" + \
              "28a226d160dad07883d04e008a7897ee" + \
              "2e4b7465d5290d0c0e6c6822236e1daa" + \
              "fb94ffe0c5da05d9476be028ad7c1d81" 

ivct_string2 ="5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253"

plaintext = decrypt(ivct_string, key_string, modes.CBC) 
print(plaintext)
print(decrypt(ivct_string2, key_string, modes.CBC))

b'Basic CBC mode encryption needs padding.\x08\x08\x08\x08\x08\x08\x08\x08'
b'Our implementation uses rand. IV\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'


In [7]:
key_string = "36f18357be4dbd77f050515c73fcf9f2"
ivct_string = "69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329"
ivct_string2 = "770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451"

print(decrypt(ivct_string, key_string, modes.CTR))
print(decrypt(ivct_string2, key_string, modes.CTR))

b'CTR mode lets you build a stream cipher from a block cipher.'
b'Always avoid the two time pad!'
